In [1]:
from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
from natsort import natsorted

# reload notebook automatically after changes to source python files
%load_ext autoreload
%autoreload 2

# change base folder to parent
import os
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(os.getcwd())

/mnt/C45ADD865ADD7620/i3d-rrc/ic-topo-nav/mickst3r


In [2]:
from mast3r_src.model import AsymmetricMASt3R
from dust3r_src.dust3r.inference import inference

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model_name = "./checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth"
model = AsymmetricMASt3R.from_pretrained(model_name).to(device)

Using device: cuda
... loading model from ./checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth
instantiating : AsymmetricMASt3R(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100',img_size=(512, 512), head_type='catmlp+dpt', output_mode='pts3d+desc24', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), patch_embed_cls='PatchEmbedDust3R', two_confs=True, desc_conf_mode=('exp', 0, inf), landscape_only=False)
<All keys matched successfully>


In [3]:
from src.datasets.utils import read_intrinsics
from dust3r_src.dust3r.utils.image import load_images

dataset_root = Path("./data/mapfree/val")

VAL_SCENES = ["s00460", "s00474", "s00482", "s00489", "s00495"]
scene_paths = [dataset_root / scene for scene in VAL_SCENES]
print(f"Found {len(scene_paths)} scenes")

scene_path = scene_paths[0]
print(f"Selected scene: {scene_path}")

intrinsics_path = scene_path / "intrinsics.txt"
K = read_intrinsics(intrinsics_path, resize=(384, 512)) # resize to model input size

reference_image_folder = scene_path / "seq0"
ref_image_path = str(reference_image_folder / "frame_00000.jpg")

DOWNSAMPLE = 20
query_image_folder = scene_path / "seq1"
query_image_paths = natsorted(list(str(p) for p in query_image_folder.glob("*.jpg")))[::DOWNSAMPLE]

BATCH_SIZE = 8
query_image_paths = query_image_paths[:BATCH_SIZE]

query_images = load_images(query_image_paths, size=512, verbose=False)
reference_image = load_images([ref_image_path], size=512, verbose=False)[0]

pairs = [(reference_image, query_image) for query_image in query_images]
pairs_rev = [(query_image, reference_image) for query_image in query_images]

Found 5 scenes
Selected scene: data/mapfree/val/s00460


In [4]:
output_1 = inference(pairs, model, device, batch_size=BATCH_SIZE, verbose=False)
output_2 = inference(pairs_rev, model, device, batch_size=BATCH_SIZE, verbose=False)

view1, pred1 = output_1['view1'], output_1['pred1']
view2, pred2 = output_2['view1'], output_2['pred1']

# extract confidence from pointmaps of both views
conf1, conf2 = pred1['conf'], pred2['conf'] # Shape (1, H_resize, W_resize)
print(f"\nConfidence shape: {conf1.shape}, {conf2.shape}")

# extract depths from pointmaps of both views as the z-coordinate
depth1, depth2 = pred1['pts3d'][..., 2], pred2['pts3d'][..., 2]
print(f"Depths shape: {depth1.shape}, {depth2.shape}")

# extract descriptors from descriptor maps of both views
desc1, desc2 = pred1['desc'], pred2['desc']
print(f"Descriptors shape: {desc1.shape}, {desc2.shape}")

# extract descriptors confidence from descriptor maps of both views
desc_conf1, desc_conf2 = pred1['desc_conf'], pred2['desc_conf']
# print(f"Descriptors confidence shape: {desc_conf1.shape}, {desc_conf2.shape}")


Confidence shape: torch.Size([8, 512, 384]), torch.Size([8, 512, 384])
Depths shape: torch.Size([8, 512, 384]), torch.Size([8, 512, 384])
Descriptors shape: torch.Size([8, 512, 384, 24]), torch.Size([8, 512, 384, 24])


In [5]:
from config.default import cfg
cfg.merge_from_file('config/prob_pose.yaml')
from src.diff_downsample_maps import downsample_maps_w_kpts

from src.diff_probabilistic_procrustes import e2eProbabilisticProcrustesSolver
from src.diff_compute_correspondences import ComputeCorrespondences

data_batch = downsample_maps_w_kpts(pred1, pred2, target_size=(51, 38), conf_type='conf', device=device)

# for K_color_0 read intrinsics of reference image and create a 8, 3, 3 tensor
K_color0 = torch.from_numpy(K[f'seq0/{Path(ref_image_path).stem}.jpg']).unsqueeze(0).to(device)
K_color0_batch = K_color0.repeat(conf1.shape[0], 1, 1)

# for K_color_1 read intrinsics of all 8 query image and create a 8, 3, 3 tensor
K_color1_batch = torch.stack([torch.from_numpy(K[f'seq1/{Path(image_q_path).stem}.jpg']) for image_q_path in query_image_paths]).to(device)

data_batch['K_color0'] = K_color0_batch
data_batch['K_color1'] = K_color1_batch

compute_Correspondences = ComputeCorrespondences(cfg, device)
e2e_Procrustes = e2eProbabilisticProcrustesSolver(cfg)

data_batch = compute_Correspondences.prepare_data(data_batch)
R_batch, t_batch, inliers_batch, _ = e2e_Procrustes.estimate_pose(
    data_batch, return_inliers=True
)

In [10]:
R = R_batch[0].unsqueeze(0).detach().cpu().numpy()
t = t_batch[0].reshape(-1).detach().cpu().numpy()
inliers = inliers_batch[0].item()

print(f"Estimated R:\n{R}\n, t={t}, inliers={inliers}")

Estimated R:
[[ 0.9082328  -0.1075771   0.4037031 ]
 [ 0.17362139  0.976099   -0.13105366]
 [-0.38005912  0.18915448  0.90546274]]
, t=[-0.84435964 -0.15628873  0.48315394], inliers=497.6474609375
